In [1]:
import pandas as pd
df = pd.read_csv('SMSSpamCollection',sep='\t',header=None,names = ['Label','SMS'])

In [2]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size = 0.2)

In [3]:
df_train.SMS = df_train.SMS.str.replace(pat='\W',repl=' ').str.lower()
df_train.SMS = df_train.SMS.str.split()

C:\Users\os_sonnh1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
vocab = []
for i in df_train.SMS:
    for v in i:
        vocab.append(v)
vocab = set(vocab)
vocab = list(vocab)

In [5]:
len(vocab)

7764

In [6]:
word_counts_per_sms = {}
for w in vocab:
    word_counts_per_sms[w] = [0] * len(df_train.SMS)

In [7]:
for idx,sms in enumerate(df_train.SMS):
    for w in sms:
        word_counts_per_sms[w][idx] += 1

In [8]:
df_word = pd.DataFrame(word_counts_per_sms)

In [9]:
fin_df = pd.concat([df_train,df_word],axis = 1)

In [10]:
fin_df = fin_df.drop(fin_df[fin_df.Label.isnull()].index)

In [11]:
fin_df = fin_df.fillna(0)
len(fin_df)

4457

In [12]:
P_ham = list(fin_df.Label.value_counts()/len(fin_df))[0]
P_spam = list(fin_df.Label.value_counts()/len(fin_df))[1]

In [13]:
df_ham_tr = df_train[df_train.Label == 'ham']
df_spam_tr = df_train[df_train.Label == 'spam']

In [14]:
ham_vocab_list = []
for i in df_ham_tr.SMS:
    for w in i:
        ham_vocab_list.append(w)
ham_vocab_set = set(ham_vocab_list)
N_ham = len(ham_vocab_list)
    
    

In [15]:
spam_vocab_list = []
for i in df_spam_tr.SMS:
    for w in i:
        spam_vocab_list.append(w)
spam_vocab_set = set(spam_vocab_list)
N_spam = len(spam_vocab_list)

In [16]:
alpha = 1
N_vocab = N_ham + N_spam

In [17]:
P_ham_dict = {}
P_spam_dict = {}
#LaPlace smoothing
for i in vocab:
    P_ham_dict[i] = (ham_vocab_list.count(i) + alpha)/(N_ham + alpha * N_vocab) 
    P_spam_dict[i] = (spam_vocab_list.count(i) + alpha)/(N_spam + alpha * N_vocab) 

In [24]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = P_spam
    p_ham_given_message = P_ham
    
    for i in message:
        if i in P_spam_dict:
            p_spam_given_message *= P_spam_dict[i]
        if i in P_ham_dict:
            p_ham_given_message *= P_ham_dict[i]
    

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return'Equal proabilities, have a human classify this!'

In [28]:
df_test['predicted'] = df_test['SMS'].apply(classify)
df_test.head()

C:\Users\os_sonnh1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Label,SMS,predicted
5217,ham,Well thats nice. Too bad i cant eat it,ham
3679,spam,Promotion Number: 8714714 - UR awarded a City ...,spam
4217,ham,Er mw im filled tuth is aight,ham
1381,ham,i dnt wnt to tlk wid u,ham
476,ham,Ok I'm gonna head up to usf in like fifteen mi...,ham


In [30]:

correct = 0
total = df_test.shape[0]
    
for row in df_test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1063
Incorrect: 52
Accuracy: 0.9533632286995516
